# 🤝 Federated Learning: Hands-On Practice

## Table of Contents
1. [Setting Up Federated Learning Environment](#practice-1-setting-up-federated-learning-environment)
2. [Simulating Multiple Hospital Clients](#practice-2-simulating-multiple-hospital-clients)
3. [Implementing FedAvg Algorithm](#practice-3-implementing-fedavg-algorithm)
4. [Adding Differential Privacy](#practice-4-adding-differential-privacy)
5. [Handling Non-IID Medical Data](#practice-5-handling-non-iid-medical-data)
6. [Secure Aggregation Basics](#practice-6-secure-aggregation-basics)
7. [Performance Comparison: Centralized vs Federated](#practice-7-performance-comparison-centralized-vs-federated)
8. [Communication Efficiency Optimization](#practice-8-communication-efficiency-optimization)

## Installing and Importing Essential Libraries

In [ ]:
# Install Flower framework (uncomment if needed)
# !pip install flwr

# Import essential libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.datasets import make_classification
import warnings
warnings.filterwarnings('ignore')

# Visualization settings
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
sns.set_style('whitegrid')

print("✅ All libraries loaded successfully!")
print("🏥 Ready for Federated Learning Practice")

---
## Practice 1: Setting Up Federated Learning Environment

### 🎯 Learning Objectives
- Understand the basic architecture of federated learning
- Create a simple client-server simulation
- Learn how data stays distributed

### 📖 Key Concepts
**Federated Learning**: Training ML models across multiple decentralized devices/servers holding local data samples, without exchanging the data itself.

In [ ]:
# 1.1 Create a simple FL environment
class FederatedLearningEnvironment:
    """Simulates a basic federated learning setup"""
    
    def __init__(self, n_clients=3):
        self.n_clients = n_clients
        self.global_model = None
        self.client_models = [None] * n_clients
        
        print(f"🌐 Federated Learning Environment Created")
        print(f"   Number of clients (hospitals): {n_clients}")
        print(f"   Central server: Ready")
        print(f"   Data sharing: Disabled (Privacy preserved!) 🔒")
    
    def initialize_global_model(self):
        """Initialize the global model"""
        self.global_model = LogisticRegression(max_iter=100, random_state=42)
        print("\n✅ Global model initialized on central server")
    
    def distribute_model(self):
        """Send global model to all clients"""
        print("\n📤 Distributing global model to all clients...")
        for i in range(self.n_clients):
            # In real FL, we'd send model parameters
            self.client_models[i] = "Model distributed"
            print(f"   ✓ Client {i+1} (Hospital {i+1}): Received model")
        print("✅ Model distribution complete!")
    
    def show_architecture(self):
        """Visualize FL architecture"""
        print("\n" + "="*50)
        print("        FEDERATED LEARNING ARCHITECTURE")
        print("="*50)
        print("\n            ☁️  Central Server")
        print("                    |")
        print("          +---------+---------+")
        print("          |         |         |")
        print("         🏥        🏥        🏥")
        print("      Hospital1  Hospital2  Hospital3")
        print("        💾         💾         💾")
        print("     Local Data Local Data Local Data")
        print("\n" + "="*50)

# Create environment
fl_env = FederatedLearningEnvironment(n_clients=3)
fl_env.show_architecture()
fl_env.initialize_global_model()
fl_env.distribute_model()

---
## Practice 2: Simulating Multiple Hospital Clients

### 🎯 Learning Objectives
- Create synthetic medical datasets for different hospitals
- Simulate data heterogeneity (Non-IID data)
- Understand why hospital data differs

### 📖 Key Concepts
**Non-IID Data**: Different hospitals have different patient demographics, disease prevalence, and protocols.

In [ ]:
# 2.1 Generate heterogeneous data for hospitals
def create_hospital_data(n_hospitals=3, samples_per_hospital=200):
    """
    Create synthetic medical data with different distributions
    for each hospital (simulating Non-IID data)
    """
    hospital_datasets = []
    
    print("🏥 Generating data for each hospital...\n")
    
    for i in range(n_hospitals):
        # Each hospital has slightly different data distribution
        # This simulates regional differences, demographics, etc.
        X, y = make_classification(
            n_samples=samples_per_hospital,
            n_features=10,
            n_informative=7,
            n_redundant=2,
            n_classes=2,
            flip_y=0.1 + i*0.05,  # Different noise levels
            class_sep=1.5 - i*0.2,  # Different separability
            random_state=42 + i*10
        )
        
        # Store data
        hospital_datasets.append((X, y))
        
        # Statistics
        class_0 = np.sum(y == 0)
        class_1 = np.sum(y == 1)
        
        print(f"Hospital {i+1}:")
        print(f"  Total patients: {len(y)}")
        print(f"  Class 0 (Healthy): {class_0} ({class_0/len(y)*100:.1f}%)")
        print(f"  Class 1 (Disease): {class_1} ({class_1/len(y)*100:.1f}%)")
        print(f"  Data distribution: {'Imbalanced' if abs(class_0-class_1) > 30 else 'Balanced'}")
        print()
    
    return hospital_datasets

# Generate data
hospital_data = create_hospital_data(n_hospitals=3, samples_per_hospital=200)

In [ ]:
# 2.2 Visualize data heterogeneity
def visualize_data_heterogeneity(hospital_data):
    """Visualize how data differs across hospitals"""
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    for i, (X, y) in enumerate(hospital_data):
        # Use first two features for visualization
        axes[i].scatter(X[y==0][:, 0], X[y==0][:, 1], 
                       alpha=0.6, label='Healthy', c='blue', s=30)
        axes[i].scatter(X[y==1][:, 0], X[y==1][:, 1], 
                       alpha=0.6, label='Disease', c='red', s=30)
        axes[i].set_title(f'Hospital {i+1} Data Distribution', fontsize=12, fontweight='bold')
        axes[i].set_xlabel('Feature 1')
        axes[i].set_ylabel('Feature 2')
        axes[i].legend()
        axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("📊 Notice how each hospital has different data patterns!")
    print("   This is called Non-IID (Non-Independently and Identically Distributed) data.")

visualize_data_heterogeneity(hospital_data)

---
## Practice 3: Implementing FedAvg Algorithm

### 🎯 Learning Objectives
- Implement the core FedAvg (Federated Averaging) algorithm
- Understand weighted aggregation
- Compare with centralized training

### 📖 Key Concepts
**FedAvg Formula**: $w_{global} = \sum_{i=1}^{n} \frac{n_i}{N} \times w_i$

where $n_i$ is the number of samples at client $i$, and $N$ is the total number of samples.

In [ ]:
# 3.1 Implement FedAvg step by step
class FedAvgTrainer:
    """Federated Averaging implementation"""
    
    def __init__(self, hospital_data):
        self.hospital_data = hospital_data
        self.n_hospitals = len(hospital_data)
        self.global_weights = None
        self.global_bias = None
        
        # Calculate dataset sizes
        self.dataset_sizes = [len(y) for _, y in hospital_data]
        self.total_samples = sum(self.dataset_sizes)
        
        print("🎯 FedAvg Trainer Initialized")
        print(f"   Total hospitals: {self.n_hospitals}")
        print(f"   Total samples: {self.total_samples}")
        print(f"   Samples per hospital: {self.dataset_sizes}")
    
    def local_training(self, hospital_id):
        """Train model locally at a hospital"""
        X, y = self.hospital_data[hospital_id]
        
        # Local model
        model = LogisticRegression(max_iter=50, random_state=42)
        model.fit(X, y)
        
        # Get trained weights
        weights = model.coef_[0]
        bias = model.intercept_[0]
        
        return weights, bias
    
    def aggregate_models(self, all_weights, all_biases):
        """
        Aggregate models using weighted averaging (FedAvg)
        """
        print("\n📊 Aggregating models using FedAvg...")
        print("   Formula: w_global = Σ(n_i/N × w_i)\n")
        
        # Initialize
        aggregated_weights = np.zeros_like(all_weights[0])
        aggregated_bias = 0.0
        
        # Weighted averaging
        for i in range(self.n_hospitals):
            weight_factor = self.dataset_sizes[i] / self.total_samples
            aggregated_weights += weight_factor * all_weights[i]
            aggregated_bias += weight_factor * all_biases[i]
            
            print(f"   Hospital {i+1}: weight = {weight_factor:.3f} "
                  f"(samples: {self.dataset_sizes[i]}/{self.total_samples})")
        
        print("\n✅ Aggregation complete!")
        return aggregated_weights, aggregated_bias
    
    def federated_round(self, round_num):
        """Execute one round of federated training"""
        print(f"\n{'='*60}")
        print(f"Round {round_num}: Federated Training")
        print(f"{'='*60}")
        
        # Step 1: Local training at each hospital
        print("\n📍 Step 1: Local Training at Each Hospital")
        all_weights = []
        all_biases = []
        
        for i in range(self.n_hospitals):
            print(f"   Training at Hospital {i+1}... ", end="")
            weights, bias = self.local_training(i)
            all_weights.append(weights)
            all_biases.append(bias)
            print("✓ Complete")
        
        # Step 2: Send updates to server
        print("\n📤 Step 2: Sending Model Updates to Server")
        print("   (Only model parameters sent, not raw data!)")
        
        # Step 3: Aggregate on server
        print("\n🔄 Step 3: Server Aggregation (FedAvg)")
        self.global_weights, self.global_bias = self.aggregate_models(
            all_weights, all_biases
        )
        
        # Step 4: Distribute back to clients
        print("\n📥 Step 4: Distributing Global Model to All Hospitals")
        print("   ✓ Model parameters updated at all hospitals")
        
        return self.global_weights, self.global_bias

# Create trainer and run one round
fedavg_trainer = FedAvgTrainer(hospital_data)
global_w, global_b = fedavg_trainer.federated_round(round_num=1)

---
## Practice 4: Adding Differential Privacy

### 🎯 Learning Objectives
- Understand differential privacy mechanisms
- Add Gaussian noise to model updates
- Balance privacy and accuracy

### 📖 Key Concepts
**Differential Privacy**: Adding calibrated noise to protect individual data points.

**Formula**: $\tilde{w} = w + \mathcal{N}(0, \sigma^2)$ where $\sigma$ is determined by privacy budget $\epsilon$

In [ ]:
# 4.1 Implement differential privacy
def add_differential_privacy(weights, bias, epsilon=1.0, sensitivity=1.0):
    """
    Add Gaussian noise for differential privacy
    
    Parameters:
    - epsilon: Privacy budget (smaller = more privacy, less accuracy)
    - sensitivity: How much one data point can change the output
    """
    print(f"\n🔒 Adding Differential Privacy")
    print(f"   Privacy budget (ε): {epsilon}")
    print(f"   Sensitivity (Δf): {sensitivity}")
    
    # Calculate noise scale
    sigma = (sensitivity * np.sqrt(2 * np.log(1.25))) / epsilon
    print(f"   Noise scale (σ): {sigma:.4f}")
    
    # Add Gaussian noise
    noisy_weights = weights + np.random.normal(0, sigma, size=weights.shape)
    noisy_bias = bias + np.random.normal(0, sigma)
    
    # Calculate noise magnitude
    noise_magnitude = np.linalg.norm(noisy_weights - weights)
    print(f"   Noise magnitude added: {noise_magnitude:.4f}")
    
    print("\n✅ Differential privacy applied!")
    print("   🎭 Individual data points are now protected")
    
    return noisy_weights, noisy_bias

# Test with different privacy levels
print("Comparing Different Privacy Levels:")
print("="*60)

# High privacy (low epsilon)
print("\n1️⃣ High Privacy (ε=0.1):")
private_w1, private_b1 = add_differential_privacy(global_w, global_b, epsilon=0.1)

# Medium privacy
print("\n2️⃣ Medium Privacy (ε=1.0):")
private_w2, private_b2 = add_differential_privacy(global_w, global_b, epsilon=1.0)

# Low privacy (high epsilon)
print("\n3️⃣ Low Privacy (ε=10.0):")
private_w3, private_b3 = add_differential_privacy(global_w, global_b, epsilon=10.0)

print("\n" + "="*60)
print("Key Insight: Lower ε = More Privacy = More Noise = Lower Accuracy")

---
## Practice 5: Handling Non-IID Medical Data

### 🎯 Learning Objectives
- Understand challenges of Non-IID data
- Implement FedProx for better handling
- Compare FedAvg vs FedProx

### 📖 Key Concepts
**FedProx**: Adds a proximal term to handle heterogeneity

**Formula**: $\min F(w) + \frac{\mu}{2}||w - w_t||^2$

In [ ]:
# 5.1 Visualize Non-IID challenges
def analyze_data_heterogeneity(hospital_data):
    """Analyze how different the hospital datasets are"""
    
    print("📊 Analyzing Data Heterogeneity\n")
    
    # Calculate statistics for each hospital
    stats = []
    for i, (X, y) in enumerate(hospital_data):
        mean_features = X.mean(axis=0)
        std_features = X.std(axis=0)
        class_ratio = y.sum() / len(y)
        
        stats.append({
            'hospital': i+1,
            'mean': mean_features.mean(),
            'std': std_features.mean(),
            'positive_ratio': class_ratio
        })
    
    # Display statistics
    df_stats = pd.DataFrame(stats)
    print(df_stats.to_string(index=False))
    
    # Calculate heterogeneity score
    mean_variance = df_stats['mean'].var()
    ratio_variance = df_stats['positive_ratio'].var()
    
    print(f"\n📈 Heterogeneity Metrics:")
    print(f"   Mean variance across hospitals: {mean_variance:.4f}")
    print(f"   Class ratio variance: {ratio_variance:.4f}")
    
    if ratio_variance > 0.01:
        print("\n⚠️  High heterogeneity detected!")
        print("   Recommendation: Use FedProx instead of FedAvg")
    else:
        print("\n✅ Low heterogeneity - FedAvg should work well")

analyze_data_heterogeneity(hospital_data)

---
## Practice 6: Secure Aggregation Basics

### 🎯 Learning Objectives
- Understand secure aggregation concept
- Implement simple masking mechanism
- Verify that individual updates remain hidden

### 📖 Key Concepts
**Secure Aggregation**: Server learns only the aggregate, not individual updates

In [ ]:
# 6.1 Simple secure aggregation simulation
def secure_aggregation_demo():
    """
    Demonstrate secure aggregation where individual updates are masked
    """
    print("🛡️ Secure Aggregation Demonstration\n")
    print("="*60)
    
    # Simulate client updates
    n_clients = 3
    update_dimension = 5
    
    client_updates = []
    client_masks = []
    
    print("Step 1: Clients Create Updates and Masks\n")
    
    for i in range(n_clients):
        # Real update
        update = np.random.randn(update_dimension)
        # Random mask
        mask = np.random.randn(update_dimension)
        
        client_updates.append(update)
        client_masks.append(mask)
        
        print(f"Client {i+1}:")
        print(f"  Update: {update[:3]}... (showing first 3 values)")
        print(f"  Mask:   {mask[:3]}...")
        print()
    
    print("\nStep 2: Clients Send Masked Updates to Server\n")
    
    masked_updates = []
    for i in range(n_clients):
        # Add mask to update
        masked = client_updates[i] + client_masks[i]
        masked_updates.append(masked)
        print(f"Client {i+1} → Server: {masked[:3]}... (masked, server can't see real update!)")
    
    print("\n🔒 Server only sees masked values!\n")
    
    print("Step 3: Server Aggregates Masked Updates\n")
    
    # Aggregate masked updates
    sum_masked = sum(masked_updates)
    print(f"Sum of masked updates: {sum_masked[:3]}...")
    
    # Calculate sum of masks (which cancels out in sum)
    sum_masks = sum(client_masks)
    print(f"Sum of masks: {sum_masks[:3]}...")
    
    print("\nStep 4: Masks Cancel Out in Aggregation!\n")
    
    # Remove masks
    final_aggregate = sum_masked - sum_masks
    
    # True aggregate (without masking)
    true_aggregate = sum(client_updates)
    
    print(f"Aggregate after mask removal: {final_aggregate[:3]}...")
    print(f"True aggregate (for verification): {true_aggregate[:3]}...")
    print(f"\n✅ Results match: {np.allclose(final_aggregate, true_aggregate)}")
    
    print("\n" + "="*60)
    print("🎉 Key Achievement: Server got correct aggregate WITHOUT seeing")
    print("   individual client updates! Privacy preserved!")

secure_aggregation_demo()

---
## Practice 7: Performance Comparison - Centralized vs Federated

### 🎯 Learning Objectives
- Compare centralized training with federated learning
- Measure accuracy retention
- Understand the privacy-performance tradeoff

### 📖 Key Concepts
**Accuracy Retention**: FL typically achieves 90-95% of centralized performance while preserving privacy

In [ ]:
# 7.1 Centralized training (baseline)
def train_centralized(hospital_data):
    """
    Train on all data centrally (what we DON'T want due to privacy)
    """
    print("🏢 Centralized Training (Baseline)\n")
    
    # Combine all data
    X_all = np.vstack([X for X, _ in hospital_data])
    y_all = np.hstack([y for _, y in hospital_data])
    
    print(f"Total samples: {len(y_all)}")
    print("⚠️  Privacy concern: All data is centralized!\n")
    
    # Split for evaluation
    X_train, X_test, y_train, y_test = train_test_split(
        X_all, y_all, test_size=0.2, random_state=42
    )
    
    # Train
    model = LogisticRegression(max_iter=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Evaluate
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"✅ Centralized Accuracy: {accuracy:.4f}")
    
    return accuracy, X_test, y_test

# 7.2 Federated training
def train_federated(hospital_data, n_rounds=3):
    """
    Train using federated learning
    """
    print("\n🤝 Federated Training\n")
    
    trainer = FedAvgTrainer(hospital_data)
    
    # Prepare test set
    X_all = np.vstack([X for X, _ in hospital_data])
    y_all = np.hstack([y for _, y in hospital_data])
    X_train, X_test, y_train, y_test = train_test_split(
        X_all, y_all, test_size=0.2, random_state=42
    )
    
    # Run federated rounds
    for round_num in range(1, n_rounds + 1):
        global_w, global_b = trainer.federated_round(round_num)
        
        # Evaluate global model
        model = LogisticRegression(max_iter=1)
        model.coef_ = global_w.reshape(1, -1)
        model.intercept_ = np.array([global_b])
        model.classes_ = np.array([0, 1])
        
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        
        print(f"\n📊 Round {round_num} Accuracy: {accuracy:.4f}")
    
    print("\n✅ Federated training complete!")
    print("🔒 Privacy preserved: Data never left hospitals!")
    
    return accuracy

# Run comparison
print("="*70)
print("        CENTRALIZED vs FEDERATED LEARNING COMPARISON")
print("="*70)

centralized_acc, X_test, y_test = train_centralized(hospital_data)
federated_acc = train_federated(hospital_data, n_rounds=3)

# Final comparison
print("\n" + "="*70)
print("📊 FINAL RESULTS")
print("="*70)
print(f"Centralized Learning: {centralized_acc:.4f} (100.0%)")
print(f"Federated Learning:   {federated_acc:.4f} ({federated_acc/centralized_acc*100:.1f}% retention)")
print(f"\nAccuracy difference: {abs(centralized_acc - federated_acc):.4f}")
print(f"\n🎉 Achievement: {federated_acc/centralized_acc*100:.1f}% of centralized performance")
print("   while keeping all data private!")

---
## Practice 8: Communication Efficiency Optimization

### 🎯 Learning Objectives
- Understand communication costs in federated learning
- Implement gradient compression
- Optimize local training epochs

### 📖 Key Concepts
**Communication Cost**: Sending model updates is expensive (1-10 GB per round)

In [ ]:
# 8.1 Analyze communication costs
def analyze_communication_cost():
    """
    Calculate and visualize communication costs
    """
    print("📡 Communication Cost Analysis\n")
    print("="*60)
    
    # Model parameters
    n_features = 10
    model_params = n_features + 1  # weights + bias
    bytes_per_param = 4  # float32
    
    # Scenario parameters
    n_clients = 3
    n_rounds = 10
    
    print(f"Model parameters: {model_params}")
    print(f"Bytes per parameter: {bytes_per_param}")
    print(f"Clients: {n_clients}")
    print(f"Training rounds: {n_rounds}\n")
    
    # Calculate costs
    size_per_update = model_params * bytes_per_param / 1024  # KB
    upload_per_round = size_per_update * n_clients  # KB
    download_per_round = size_per_update * n_clients  # KB
    total_per_round = upload_per_round + download_per_round  # KB
    total_training = total_per_round * n_rounds  # KB
    
    print("💾 Communication Costs:")
    print(f"   Size per model update: {size_per_update:.2f} KB")
    print(f"   Upload per round (all clients): {upload_per_round:.2f} KB")
    print(f"   Download per round (all clients): {download_per_round:.2f} KB")
    print(f"   Total per round: {total_per_round:.2f} KB")
    print(f"   Total for training: {total_training:.2f} KB = {total_training/1024:.2f} MB")
    
    # Optimization strategies
    print("\n" + "="*60)
    print("💡 Communication Efficiency Strategies:")
    print("="*60)
    
    print("\n1️⃣ Gradient Compression (10-100x reduction)")
    compressed_size = size_per_update * 0.1  # 10x compression
    print(f"   Compressed update size: {compressed_size:.2f} KB")
    print(f"   Savings: {(1 - 0.1)*100:.0f}%")
    
    print("\n2️⃣ More Local Epochs (Reduce communication rounds)")
    print("   E=1 (1 local epoch): 10 rounds = 10 communications")
    print("   E=5 (5 local epochs): 2 rounds = 2 communications")
    print(f"   Savings: {(1 - 2/10)*100:.0f}%")
    
    print("\n3️⃣ Model Compression (Pruning/Distillation)")
    print("   Original model: 10 parameters")
    print("   Compressed model: 5 parameters (50% pruning)")
    print(f"   Savings: 50%")
    
    # Visualize
    strategies = ['Original', 'Gradient\nCompression', 'Fewer Rounds\n(Local Epochs)', 'Model\nPruning']
    costs = [total_training, total_training*0.1, total_training*0.2, total_training*0.5]
    
    plt.figure(figsize=(10, 6))
    bars = plt.bar(strategies, costs, color=['red', 'orange', 'green', 'blue'], alpha=0.7)
    plt.ylabel('Total Communication (KB)', fontsize=12)
    plt.title('Communication Cost: Original vs Optimized Strategies', 
              fontsize=14, fontweight='bold')
    plt.grid(axis='y', alpha=0.3)
    
    # Add value labels
    for i, (bar, cost) in enumerate(zip(bars, costs)):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                f'{cost:.1f} KB', ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print("\n✅ By combining strategies, we can reduce communication by 90%+!")

analyze_communication_cost()

---
## 🎯 Practice Complete!

### Summary of What We Learned:

1. **FL Architecture**: Client-server setup with distributed data
2. **Non-IID Data**: Different hospitals have different data distributions
3. **FedAvg Algorithm**: Weighted averaging - $(X^T X)^{-1} X^T y$
4. **Differential Privacy**: Adding noise to protect individual data
5. **Secure Aggregation**: Server sees only aggregates, not individual updates
6. **Performance**: FL achieves 90-95% of centralized accuracy
7. **Communication Efficiency**: Compression and local epochs reduce costs by 90%+

### Key Insights:

✅ **Privacy Preserved**: Data never leaves hospitals  
✅ **Good Performance**: 90-95% accuracy retention  
✅ **Practical**: Real deployments in COVID-19 research (20+ countries, 100+ hospitals)  
✅ **Compliant**: GDPR/HIPAA compatible with proper design

### Next Steps:

1. Try with real medical datasets (MIMIC-III, eICU)
2. Implement FedProx for better heterogeneity handling
3. Explore production frameworks (Flower, PySyft, NVIDIA FLARE)
4. Read key papers: FedAvg (McMahan et al., 2017), FedProx (Li et al., 2020)

### 📚 Resources:

- **Flower Framework**: https://flower.dev/
- **PySyft**: https://github.com/OpenMined/PySyft
- **NVIDIA FLARE**: https://nvidia.github.io/NVFlare/

---

## 🎉 Congratulations!

You've completed the Federated Learning hands-on practice!

You now understand:
- How federated learning works
- How to implement FedAvg
- How to add privacy protection
- How to optimize communication
- The tradeoffs between privacy and performance

**Keep learning and building privacy-preserving AI! 🚀**